In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.linear_model import Ridge
from matplotlib.colors import Normalize
import networkx as nx
from scipy.signal import welch
import matplotlib.cm as cm
import seaborn as sns
import torch
import torch.nn as nn
from torch.optim import Adam
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
# import neurokit2 as nk
import matplotlib.pyplot as plt
import numpy as np
import os
import json

## LSTM

In [2]:
def lorenz_derivatives(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    """Compute time derivatives [dx/dt, dy/dt, dz/dt] for the Lorenz system."""
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    """
    Numerically integrate Lorenz equations x'(t), y'(t), z'(t) using odeint.
    Returns:
       t_vals: array of time points
       sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
    """
    num_steps = int(tmax / dt) + 1
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_derivatives, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [3]:
chosen_system = "Lorenz"  # Options: "lorenz", "rossler", "chen", "chua"
tmax = 250.0
dt = 0.02
t_vals, lorenz_traj = generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=tmax,
    dt=dt,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
)

# Discard first 2,000 points as washout
washout = 2000
t_vals = t_vals[washout:]
lorenz_trajectory = lorenz_traj[washout:]

T_data = len(lorenz_traj)
print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

train_frac = 0.7
train_end = int(train_frac*(T_data-1))
# train_end = 4500
train_input  = lorenz_traj[:train_end]
train_target = lorenz_traj[1:train_end+1]
test_input   = lorenz_traj[train_end:-1]
test_target  = lorenz_traj[train_end+1:]

y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps)*dt
print(f"Train size: {len(train_input)}  Test size: {len(test_input)}")

Data length: 12501, from t=0..250.0 with dt=0.02.
Train size: 8750  Test size: 3750


In [4]:
class LSTMBaseline3D:
    """
    Lightweight single-layer LSTM for 3-dim Lorenz forecasting.
    * hidden_size=32 → ~4.8k trainable parameters
    * fit() trains in teacher-forcing mode
    * predict() produces autoregressive roll-out
    """

    def __init__(self,
                 input_dim:  int = 3,
                 hidden_size: int = 32,
                 output_dim: int = 3,
                 lr: float = 1e-3,
                 epochs: int = 30,
                 device: str = 'cpu',
                 seed: int = 0):
        torch.manual_seed(seed); np.random.seed(seed)

        self.device  = torch.device(device)
        self.epochs  = epochs
        self.model   = nn.LSTM(input_dim, hidden_size,
                               batch_first=True).to(self.device)
        self.head    = nn.Linear(hidden_size, output_dim).to(self.device)
        self.crit    = nn.MSELoss()
        self.optim   = Adam(list(self.model.parameters())+
                            list(self.head.parameters()), lr=lr)

    # ---------------------------------------------------------
    @torch.no_grad()
    def _init_hidden(self, batch_sz=1):
        h0 = torch.zeros(1, batch_sz,
                         self.model.hidden_size,
                         device=self.device)
        c0 = torch.zeros_like(h0)
        return (h0, c0)

    # ---------------------------------------------------------
    def fit(self, x_np: np.ndarray, y_np: np.ndarray):
        """
        x_np shape [T, 3]  (input  at t)
        y_np shape [T, 3]  (target at t)
        """
        x = torch.tensor(x_np, dtype=torch.float32,
                         device=self.device).unsqueeze(0)  # [1,T,3]
        y = torch.tensor(y_np, dtype=torch.float32,
                         device=self.device).unsqueeze(0)

        for _ in range(self.epochs):
            self.optim.zero_grad()
            out, _ = self.model(x, self._init_hidden())
            pred   = self.head(out)
            loss   = self.crit(pred, y)
            loss.backward()
            self.optim.step()

    # ---------------------------------------------------------
    @torch.no_grad()
    def predict(self, init_u: np.ndarray, n_steps: int):
        """
        Autoregressive roll-out.
        init_u : initial 3-vector (last known sample)
        Returns array of shape [n_steps, 3].
        """
        self.model.eval(); self.head.eval()

        inp     = torch.tensor(init_u[None, None, :],
                               dtype=torch.float32, device=self.device)
        h, c    = self._init_hidden()
        preds   = np.empty((n_steps, 3), dtype=np.float32)

        for t in range(n_steps):
            out, (h, c) = self.model(inp, (h, c))
            y           = self.head(out)
            preds[t]    = y.squeeze(0).cpu().numpy()
            inp         = y.detach()    # feed prediction back

        return preds
    
    @torch.no_grad()
    def predict_open_loop(self, x_np: np.ndarray):
        """
        Open-loop prediction using teacher-forced inputs (like during training).
        x_np shape: [T, 3] – input sequence
        Returns:
            preds: [T, 3] – predicted output sequence
        """
        self.model.eval(); self.head.eval()

        x = torch.tensor(x_np, dtype=torch.float32,
                         device=self.device).unsqueeze(0)  # [1, T, 3]
        out, _ = self.model(x, self._init_hidden())
        preds = self.head(out).squeeze(0).cpu().numpy()  # [T, 3]

        return preds


# lstm_baseline = LSTMBaseline3D(
#                     hidden_size=38,         # parameter budget ~ 4800
#                     lr=1e-3,
#                     epochs=100,
#                     device='cuda' if torch.cuda.is_available() else 'cpu',
#                     seed=45)
# lstm_baseline.fit(train_input, train_target)

# # one-step roll-out to build an initial vector for auto-regressive mode
# init_vec = train_target[-1]                # last teacher-forced target
# lstm_preds = lstm_baseline.predict(init_vec,
#                                    n_steps=len(test_input))
# lstm_preds_open_loop = lstm_baseline.predict_open_loop(test_input)


In [5]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

### MIT-BIH

In [6]:
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb
import wfdb

# Download and load record and annotations for patient #100
record = wfdb.rdrecord('100', sampfrom=0, sampto=25002, pn_dir='mitdb')  # first 20,000 samples
annotation = wfdb.rdann('100', 'atr', sampfrom=0, sampto=25002, pn_dir='mitdb')
# Get input signal u(t) from the first channel
u = record.p_signal[:, 0] 
u
# Normalize input
u_min = np.min(u)
u_max = np.max(u)
u_norm = (u - u_min) / (u_max - u_min)
fs = record.fs  # sampling frequency (should be 360 Hz)
t_vals = np.arange(len(u_norm)) / fs
emb_dim = 3
# inputs = u_norm
inputs = create_delay_embedding(u_norm, emb_dim)

# Create target array (heartbeat locations)
targets = np.zeros(len(u_norm))
targets[annotation.sample] = 1  # mark annotations as 1 (heartbeat)
targets = create_delay_embedding(targets, emb_dim)
data_size = len(inputs)
train_size = 15000
train_input = inputs[:train_size]
train_target = targets[:train_size]
test_input = inputs[train_size+1:]
test_target = targets[train_size+1:]
test_size = len(test_input)
print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

Total samples: 25000, train size: 15000, test size: 9999


In [7]:
horizons = list(range(1, 1001))

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    lstm_baseline = LSTMBaseline3D(
                        hidden_size=32,         # parameter budget ~ 4800
                        lr=1e-3,
                        epochs=400,
                        device='cuda' if torch.cuda.is_available() else 'cpu',
                        seed=seed)
    lstm_baseline.fit(train_input, train_target)

    # one-step roll-out to build an initial vector for auto-regressive mode
    init_vec = train_target[-1]                # last teacher-forced target
    lstm_preds = lstm_baseline.predict_open_loop(test_input)

    nrmse = evaluate_nrmse(lstm_preds, test_target, horizons)
    nrmse_dict['LSTM'].append(nrmse)
    # for horizon, value in nrmse.items():
    #     nrmse_dict[horizon].append(value)

KeyboardInterrupt: 

In [15]:
nrmse_dict

defaultdict(list,
            {'LSTM': [{1: array([inf, inf, inf]),
               2: array([inf, inf, inf]),
               3: array([inf, inf, inf]),
               4: array([inf, inf, inf]),
               5: array([inf, inf, inf]),
               6: array([inf, inf, inf]),
               7: array([inf, inf, inf]),
               8: array([inf, inf, inf]),
               9: array([       inf,        inf, 0.97818592]),
               10: array([       inf, 1.30358392, 1.30358392]),
               11: array([1.53800479, 1.53800479, 1.53800479]),
               12: array([1.55685543, 1.55685543, 1.55685543]),
               13: array([1.55529061, 1.55529061, 1.55529061]),
               14: array([1.55348434, 1.55348434, 1.55348434]),
               15: array([1.55246515, 1.55246515, 1.55246515]),
               16: array([1.55100231, 1.55100231, 1.55100231]),
               17: array([1.54906643, 1.54906643, 1.54906643]),
               18: array([1.54688378, 1.54688378, 1.54688378]),

In [14]:
with open('lstmmitbih.json', 'w') as f:
    json.dump(nrmse_dict, f)

TypeError: Object of type ndarray is not JSON serializable

In [12]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'LSTM':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['LSTM']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
LSTM             
--------------------------------------------------------------------------------------------------------------------------------------------
300        1.403689251708608 ± 0.11595694894078283
600        1.3572118807998188 ± 0.11991708060885164
1000       1.3587061311672828 ± 0.11065892082379176


### Sunspot (Monthly)

In [58]:
import pandas as pd
file_path = 'RealWorld/datasets/SN_m_tot_V2.0.csv'

df = pd.read_csv(file_path, sep=';', header = None)
df
data = df.iloc[:, 3].values
dt = 1
dataset_size = len(data)
data = create_delay_embedding(data, 3)
print(f"Dataset size: {dataset_size}")

# Train/Test Split
train_end = 2000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}\nTest size: {len(test_input)}")

Dataset size: 3315
Train size: 2000
Test size: 1312


In [63]:
lstm_baseline = LSTMBaseline3D(
                    hidden_size=32,         # parameter budget ~ 4800
                    lr=1e-3,
                    epochs=400,
                    device='cuda' if torch.cuda.is_available() else 'cpu',
                    seed=42)
lstm_baseline.fit(train_input, train_target)

# one-step roll-out to build an initial vector for auto-regressive mode
init_vec = train_target[-1]                # last teacher-forced target
lstm_preds = lstm_baseline.predict_open_loop(test_input)

nrmse = evaluate_nrmse(lstm_preds, test_target, horizons=[200, 400, 600, 800, 1000])

In [64]:
nrmse

{200: array([2.77286289, 2.76831341, 2.75809817]),
 400: array([2.64045184, 2.61904885, 2.60626304]),
 600: array([2.5336566 , 2.53202015, 2.5302861 ]),
 800: array([2.62882725, 2.62769681, 2.62751743]),
 1000: array([2.5902262, 2.5902443, 2.5904081])}

In [65]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    lstm_baseline = LSTMBaseline3D(
                        hidden_size=32,         # parameter budget ~ 4800
                        lr=1e-3,
                        epochs=400,
                        device='cuda' if torch.cuda.is_available() else 'cpu',
                        seed=seed)
    lstm_baseline.fit(train_input, train_target)

    # one-step roll-out to build an initial vector for auto-regressive mode
    init_vec = train_target[-1]                # last teacher-forced target
    lstm_preds = lstm_baseline.predict_open_loop(test_input)

    nrmse = evaluate_nrmse(lstm_preds, test_target, horizons)
    nrmse_dict['LSTM'].append(nrmse)
    # for horizon, value in nrmse.items():
    #     nrmse_dict[horizon].append(value)

In [66]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'LSTM':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['LSTM']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
LSTM             
--------------------------------------------------------------------------------------------------------------------------------------------
300        2.675008492187419 ± 0.012923192834715486
600        2.537752244792846 ± 0.009405482742044077
1000       2.5961442773004455 ± 0.009656027200575885


### Santa Fe

In [68]:
file_path = 'RealWorld/datasets/santa-fe-time-series-competition-data-set-b-1.0.0/b1.txt'

df = pd.read_csv(file_path, header=None, sep=' ')
df
# Normalize the first column (column 0) of the DataFrame
df[0] = (df[0] - df[0].min()) / (df[0].max() - df[0].min())
data = df.iloc[:, 0].values
chosen_system = "SantaFe"
dt = 1
T_data = len(data)
data = create_delay_embedding(data, 3)
print(f"Data length: {T_data}.")

# Train/Test Split
train_end = 7000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}  \nTest size: {len(test_input)}")


Data length: 17000.
Train size: 7000  
Test size: 9997


In [69]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:
    lstm_baseline = LSTMBaseline3D(
                        hidden_size=32,         # parameter budget ~ 4800
                        lr=1e-3,
                        epochs=400,
                        device='cuda' if torch.cuda.is_available() else 'cpu',
                        seed=seed)
    lstm_baseline.fit(train_input, train_target)

    # one-step roll-out to build an initial vector for auto-regressive mode
    init_vec = train_target[-1]                # last teacher-forced target
    lstm_preds = lstm_baseline.predict_open_loop(test_input)

    nrmse = evaluate_nrmse(lstm_preds, test_target, horizons)
    nrmse_dict['LSTM'].append(nrmse)
    # for horizon, value in nrmse.items():
    #     nrmse_dict[horizon].append(value)

In [70]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'LSTM':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['LSTM']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
LSTM             
--------------------------------------------------------------------------------------------------------------------------------------------
300        1.0224924845401087 ± 0.14157896135439874
600        0.7022846945297287 ± 0.08675457054383437
1000       0.7800291094963395 ± 0.09463118835634546


## TCN

In [4]:
class TCNBaseline3D(nn.Module):
    """
    2-layer causal TCN       (kernel=3, dilation=1 & 2, padding chosen
    so receptive field = 5 time-steps).
    ----------------------
    • input_dim  = 3
    • hidden_dim = 32  → total ≈ 4.9 k parameters
    • output_dim = 3    (one-step prediction)
    """
    def __init__(self,
                 input_dim:  int = 3,
                 hidden_dim: int = 32,
                 output_dim: int = 3,
                 lr: float = 1e-3,
                 epochs: int = 40,
                 device: str = "cpu",
                 seed: int = 0):
        super().__init__()
        torch.manual_seed(seed); np.random.seed(seed)

        k = 3  # kernel
        # layer 1: dilation 1  → pad 2 to keep length
        self.conv1 = nn.Conv1d(input_dim, hidden_dim,
                               kernel_size=k,
                               dilation=1,
                               padding=2,
                               bias=True)
        # layer 2: dilation 2  → pad 4
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim,
                               kernel_size=k,
                               dilation=2,
                               padding=4,
                               bias=True)
        self.relu  = nn.ReLU()
        self.head  = nn.Conv1d(hidden_dim, output_dim,
                               kernel_size=1, bias=True)

        self.lr, self.epochs = lr, epochs
        self.to(device)
        self.optim = Adam(self.parameters(), lr=lr)
        self.crit  = nn.MSELoss()

    # ---------------------------------------------------------
    def forward(self, x):
        """
        x shape  [B, T, 3]  (batch, time, channels)
        return  [B, T, 3]
        """
        # reshape to Conv1d convention: (B, C, T)
        x = x.permute(0, 2, 1)
        y = self.conv1(x); y = self.relu(y[:, :, :-2])     # remove look-ahead pad
        y = self.conv2(y); y = self.relu(y[:, :, :-4])     # remove look-ahead pad
        out = self.head(y).permute(0, 2, 1)                # back to (B,T,C)
        return out

    # ---------------------------------------------------------
    def fit(self, x_np: np.ndarray, y_np: np.ndarray):
        """
        Teacher-forcing on entire sequence (batch size = 1).
        x_np, y_np shape [T, 3]
        """
        x = torch.tensor(x_np[None], dtype=torch.float32, device=next(self.parameters()).device)
        y = torch.tensor(y_np[None], dtype=torch.float32, device=next(self.parameters()).device)

        for _ in range(self.epochs):
            self.optim.zero_grad()
            pred = self.forward(x)
            loss = self.crit(pred[:, :-1], y[:, 1:])  # predict next step
            loss.backward()
            self.optim.step()

    # ---------------------------------------------------------
    @torch.no_grad()
    def predict(self, init_window: np.ndarray, n_steps: int):
        """
        Autoregressive roll-out.
        init_window : length L≥5, shape [L,3] (latest samples, earliest first)
        Returns      : [n_steps,3]
        """
        device = next(self.parameters()).device
        window = init_window.copy()
        preds  = np.empty((n_steps, 3), dtype=np.float32)

        for t in range(n_steps):
            inp = torch.tensor(window[None], dtype=torch.float32, device=device)
            y   = self.forward(inp)[0, -1].cpu().numpy()
            preds[t] = y
            window   = np.vstack([window[1:], y])  # slide window

        return preds
    
    @torch.no_grad()
    def predict_open_loop(self, x_np: np.ndarray):
        """
        Open-loop prediction (teacher-forced inputs).
        x_np shape: [T, 3]
        Returns:
            preds: [T - 1, 3] – one-step-ahead predictions
        """
        self.eval()
        device = next(self.parameters()).device

        x = torch.tensor(x_np[None], dtype=torch.float32, device=device)  # [1, T, 3]
        preds = self.forward(x)  # [1, T, 3]
        preds = preds[:, :-1]    # predict from t=0 to t=T-2 for target t=1 to t=T-1

        return preds.squeeze(0).cpu().numpy()



tcn = TCNBaseline3D(hidden_dim=32, epochs=50, lr=1e-3, device="cpu", seed=46)
tcn.fit(train_input, train_target)

# initial window must be >4 samples:
init_win = test_input[:5].copy()
tcn_preds = tcn.predict(init_win, n_steps=len(test_target))
tcn_preds_open_loop = tcn.predict_open_loop(test_input)

In [10]:
nrmse = evaluate_nrmse(tcn_preds, y_test, horizons=[200, 400, 600, 800, 1000])

In [11]:
nrmse

{200: array([3.3622266 , 2.92286399, 2.94476045]),
 400: array([3.35475998, 2.8935311 , 2.71048812]),
 600: array([3.59476628, 3.17971057, 3.2000023 ]),
 800: array([3.6408971 , 3.18923543, 3.18111239]),
 1000: array([3.62186296, 3.17139968, 3.19773441])}

### MIT-BIH

In [12]:
# MIT-BIH Dataset
def create_delay_embedding(signal, embed_dim):
    L = len(signal) - embed_dim + 1
    emb = np.zeros((L, embed_dim))
    for i in range(L):
        emb[i, :] = signal[i:i+embed_dim]
    return emb

import wfdb

# Download and load record and annotations for patient #100
record = wfdb.rdrecord('100', sampfrom=0, sampto=25002, pn_dir='mitdb')  # first 20,000 samples
annotation = wfdb.rdann('100', 'atr', sampfrom=0, sampto=25002, pn_dir='mitdb')
# Get input signal u(t) from the first channel
u = record.p_signal[:, 0] 
u
# Normalize input
u_min = np.min(u)
u_max = np.max(u)
u_norm = (u - u_min) / (u_max - u_min)
fs = record.fs  # sampling frequency (should be 360 Hz)
t_vals = np.arange(len(u_norm)) / fs
emb_dim = 3
# inputs = u_norm
inputs = create_delay_embedding(u_norm, emb_dim)

# Create target array (heartbeat locations)
targets = np.zeros(len(u_norm))
targets[annotation.sample] = 1  # mark annotations as 1 (heartbeat)
targets = create_delay_embedding(targets, emb_dim)
data_size = len(inputs)
train_size = 15000
train_input = inputs[:train_size]
train_target = targets[:train_size]
test_input = inputs[train_size+1:]
test_target = targets[train_size+1:]
test_size = len(test_input)
print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

Total samples: 25000, train size: 15000, test size: 9999


In [13]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:  
    tcn = TCNBaseline3D(hidden_dim=32, epochs=50, lr=1e-3, device="cpu", seed=seed)
    tcn.fit(train_input, train_target)

    # initial window must be >4 samples:
    # init_win = test_input[:5].copy()    
    tcn_preds = tcn.predict_open_loop(test_input)
    nrmse = evaluate_nrmse(tcn_preds, test_target, horizons)
    nrmse_dict['TCN'].append(nrmse)

In [14]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'TCN':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['TCN']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
TCN              
--------------------------------------------------------------------------------------------------------------------------------------------
300        1.620596824802384 ± 0.04332837192484439
600        1.6186667098390075 ± 0.04171223835818487
1000       1.6169760364489023 ± 0.04015689973379543


In [25]:
import pandas as pd
file_path = 'RealWorld/datasets/SN_m_tot_V2.0.csv'

df = pd.read_csv(file_path, sep=';', header = None)
df
data = df.iloc[:, 3].values
dt = 1
dataset_size = len(data)
data = create_delay_embedding(data, 3)
print(f"Dataset size: {dataset_size}")

# Train/Test Split
train_end = 2000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}\nTest size: {len(test_input)}")

Dataset size: 3315
Train size: 2000
Test size: 1312


In [26]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:  
    tcn = TCNBaseline3D(hidden_dim=32, epochs=40, lr=1e-3, device="cpu", seed=seed)
    tcn.fit(train_input, train_target)

    # initial window must be >4 samples:
    # init_win = test_input[:5].copy()    
    tcn_preds = tcn.predict_open_loop(test_input)
    nrmse = evaluate_nrmse(tcn_preds, test_target, horizons)
    nrmse_dict['TCN'].append(nrmse)

In [27]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'TCN':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['TCN']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
TCN              
--------------------------------------------------------------------------------------------------------------------------------------------
300        0.920251233826814 ± 0.051167505977109125
600        0.7670735717539632 ± 0.05024783059948199
1000       0.7532650172790911 ± 0.0496774645230772


In [18]:
file_path = 'RealWorld/datasets/santa-fe-time-series-competition-data-set-b-1.0.0/b1.txt'

df = pd.read_csv(file_path, header=None, sep=' ')
df
# Normalize the first column (column 0) of the DataFrame
df[0] = (df[0] - df[0].min()) / (df[0].max() - df[0].min())
data = df.iloc[:, 0].values
chosen_system = "SantaFe"
dt = 1
T_data = len(data)
data = create_delay_embedding(data, 3)
print(f"Data length: {T_data}.")

# Train/Test Split
train_end = 7000
train_input  = data[:train_end]
train_target = data[1:train_end+1]
test_input   = data[train_end:-1]
test_target  = data[train_end+1:]
y_test = test_target
n_test_steps = len(test_target)
time_test = np.arange(n_test_steps) * dt

print(f"Train size: {len(train_input)}  \nTest size: {len(test_input)}")


Data length: 17000.
Train size: 7000  
Test size: 9997


In [23]:
horizons = [300, 600, 1000]

nrmse_dict = defaultdict(list)
seeds = range(995, 1025)

for seed in seeds:  
    tcn = TCNBaseline3D(hidden_dim=32, epochs=100, lr=1e-3, device="cpu", seed=seed)
    tcn.fit(train_input, train_target)

    # initial window must be >4 samples:
    # init_win = test_input[:5].copy()    
    tcn_preds = tcn.predict_open_loop(test_input)
    nrmse = evaluate_nrmse(tcn_preds, test_target, horizons)
    nrmse_dict['TCN'].append(nrmse)

In [24]:
print("\nNRMSE for Different Prediction Horizons:")
print("-" * 140)
print(f"{'TCN':<17}")
print("-" * 140)

for horizon in horizons:
    hfr_vals = [np.mean(hfr_nrmse[horizon]) for hfr_nrmse in nrmse_dict['TCN']]

    print(f"{horizon:<10}", end=" ")
    for vals in [hfr_vals]:
        mean = np.mean(vals)
        std = np.std(vals)
        print(f"{mean} ± {std}".ljust(18), end="")
    print()


NRMSE for Different Prediction Horizons:
--------------------------------------------------------------------------------------------------------------------------------------------
TCN              
--------------------------------------------------------------------------------------------------------------------------------------------
300        1.2064525351486612 ± 0.18056236093445113
600        0.8929744613975146 ± 0.12518798413065568
1000       0.9639359904495735 ± 0.13378513242802026
